# Paddyfield Phenology Analysis with S1+S2 Fusion (sits + FuseTS MOGPR)

This notebook demonstrates an advanced workflow for extracting paddyfield phenology metrics using **multi-sensor fusion**:
- **R sits package** for Sentinel-1 SAR + Sentinel-2 optical data acquisition
- **FuseTS MOGPR** for intelligent sensor fusion and gap-filling
- **Phenology extraction** from gap-filled NDVI

## Why S1 + S2 Fusion?

**Sentinel-2 (optical)**:
- ✅ Direct vegetation measurement (NDVI)
- ❌ Cloud contamination → gaps in time series
- ❌ Limited in monsoon/rainy seasons

**Sentinel-1 (SAR)**:
- ✅ All-weather capability (penetrates clouds)
- ✅ Sensitive to water/soil moisture (good for rice flooding)
- ❌ Indirect vegetation signal

**MOGPR Fusion**:
- ✅ Uses S1-S2 correlations to fill NDVI gaps
- ✅ Produces continuous, gap-free time series
- ✅ Better phenology detection in cloudy regions

## Workflow Overview

1. **Data Extraction (R)**: Use sits to get S1 (VV, VH) + S2 (NDVI) time series
2. **Export to Python**: Convert sits data to FuseTS-compatible format
3. **MOGPR Fusion (Python)**: Fuse S1+S2 for gap-free NDVI
4. **Phenology Extraction (Python)**: Calculate SOS, EOS from fused NDVI
5. **Comparison**: Evaluate fusion benefits vs S2-only approach
6. **Visualization**: Create comprehensive plots and maps

## Prerequisites

### R packages:
```r
install.packages(c("sits", "sf", "terra", "dplyr"))
```

### Python packages:
```bash
pip install fusets rioxarray matplotlib pandas geopandas GPy
```

---
# PART 1: MULTI-SENSOR DATA EXTRACTION WITH SITS (R)

**Note**: Run this section in R. Results will be loaded in Python below.

---

## Step 1.1: Setup and Study Area Definition (R)

In [ ]:
%%R
library(sits)
library(sf)
library(terra)
library(dplyr)

# Source FuseTS helper functions
source("/home/unika_sianturi/work/FuseTS/scripts/sits_to_fusets.R")

# Define study area (example: Demak region, Indonesia)
# Option A: Load from file
study_area <- st_read("/path/to/demak_boundary.shp")

# Option B: Create from coordinates
# study_area <- st_bbox(c(xmin = 110.5, ymin = -6.95, 
#                         xmax = 110.8, ymax = -6.75), 
#                       crs = 4326) %>% st_as_sfc() %>% st_as_sf()

# Define temporal period
start_date <- "2024-01-01"
end_date <- "2024-12-31"

print(study_area)

## Step 1.2: Get Sentinel-2 Data Cube (R)

In [ ]:
%%R
# Get Sentinel-2 cube from Microsoft Planetary Computer
s2_cube <- sits_cube(
  source = "MPC",
  collection = "SENTINEL-2-L2A",
  roi = study_area,
  start_date = start_date,
  end_date = end_date,
  bands = c("B04", "B08")  # Red, NIR for NDVI
)

# Calculate NDVI
s2_ndvi <- sits_apply(s2_cube,
  NDVI = (B08 - B04) / (B08 + B04)
)

print(s2_ndvi)

## Step 1.3: Get Sentinel-1 Data Cube (R)

In [ ]:
%%R
# Get Sentinel-1 GRD cube from Microsoft Planetary Computer
s1_cube <- sits_cube(
  source = "MPC",
  collection = "SENTINEL-1-GRD",
  roi = study_area,
  start_date = start_date,
  end_date = end_date,
  bands = c("VV", "VH")  # Both polarizations
)

print(s1_cube)
print(paste("S1 observations:", nrow(s1_cube)))
print(paste("S2 observations:", nrow(s2_ndvi)))

## Step 1.4: Export S1 + S2 Data for FuseTS (R)

Choose one of two options:
- **Option A**: Point-based (faster, recommended for first try)
- **Option B**: Raster-based (full spatial coverage)

### Option A: Point-Based Extraction

In [ ]:
%%R
# Sample points across study area
sample_points <- st_sample(study_area, size = 200)

# Alternative: Use existing paddyfield points
# sample_points <- st_read("/path/to/paddyfield_points.shp")

# Extract S1 time series
s1_samples <- sits_get_data(s1_cube, samples = sample_points)

# Extract S2 time series
s2_samples <- sits_get_data(s2_ndvi, samples = sample_points)

# Export S1 data
sits_to_fusets_csv(s1_samples, 
                   "/home/unika_sianturi/work/FuseTS/data/s1_timeseries.csv",
                   bands = c("VV", "VH"))

# Export S2 NDVI data
sits_to_fusets_csv(s2_samples, 
                   "/home/unika_sianturi/work/FuseTS/data/s2_ndvi_timeseries.csv",
                   bands = c("NDVI"))

print(paste("Exported S1 data:", nrow(s1_samples), "locations"))
print(paste("Exported S2 data:", nrow(s2_samples), "locations"))

### Option B: Raster-Based Extraction (Full Spatial Coverage)

In [ ]:
%%R
# Export S1 cube as GeoTIFF stacks
sits_cube_to_fusets_geotiff(
  s1_cube,
  output_dir = "/home/unika_sianturi/work/FuseTS/data/s1_output",
  bands = c("VV", "VH")
)

# Export S2 NDVI cube as GeoTIFF stack
sits_cube_to_fusets_geotiff(
  s2_ndvi,
  output_dir = "/home/unika_sianturi/work/FuseTS/data/s2_output",
  bands = c("NDVI")
)

print("Exported S1 GeoTIFF stacks to data/s1_output/")
print("Exported S2 GeoTIFF stacks to data/s2_output/")
print("S1 files:")
list.files("/home/unika_sianturi/work/FuseTS/data/s1_output", pattern = ".tif$")
print("S2 files:")
list.files("/home/unika_sianturi/work/FuseTS/data/s2_output", pattern = ".tif$")

---
# PART 2: MOGPR FUSION & PHENOLOGY EXTRACTION (Python)
---

## Step 2.1: Setup Python Environment

In [ ]:
import sys
sys.path.insert(0, '/home/unika_sianturi/work/FuseTS/src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import warnings
warnings.filterwarnings('ignore')

from fusets.io.sits_bridge import load_sits_csv, load_sits_geotiff, prepare_mogpr_format
from fusets.mogpr import MOGPRTransformer
from fusets import whittaker
from fusets.analytics import phenology

print("FuseTS imported successfully!")
print("MOGPR fusion enabled ✓")

## Step 2.2: Load S1 and S2 Data from sits

Choose the option that matches your R export:

### Option A: Load Point Time Series

In [ ]:
# Load S1 time series from Klambu-Glapan extraction
# NOTE: Update these paths based on where you ran the R extraction script
# If you used extract_sits_klambu_glapan.R, the files are in sits_exports/

s1_data = load_sits_csv(
    "/home/unika_sianturi/work/FuseTS/data/sits_exports/s1_klambu_glapan_timeseries.csv",
    time_col='Index',
    band_cols=['VV', 'VH']
)

# Load S2 NDVI time series
s2_data = load_sits_csv(
    "/home/unika_sianturi/work/FuseTS/data/sits_exports/s2_klambu_glapan_ndvi_timeseries.csv",
    time_col='Index',
    band_cols=['NDVI']
)

print("S1 data loaded:")
print(f"  Dimensions: {s1_data.dims}")
print(f"  Variables: {list(s1_data.data_vars)}")
print(f"  Time points: {len(s1_data.t)}")

print("\nS2 data loaded:")
print(f"  Dimensions: {s2_data.dims}")
print(f"  Variables: {list(s2_data.data_vars)}")
print(f"  Time points: {len(s2_data.t)}")

### Option B: Load Raster Time Series Stacks

In [ ]:
# Load S1 raster stacks (if using Option B from R)
import rioxarray as rxr

s1_vv_stack = load_sits_geotiff("/home/unika_sianturi/work/FuseTS/data/s1_output/VV_stack.tif")
s1_vh_stack = load_sits_geotiff("/home/unika_sianturi/work/FuseTS/data/s1_output/VH_stack.tif")
s2_ndvi_stack = load_sits_geotiff("/home/unika_sianturi/work/FuseTS/data/s2_output/NDVI_stack.tif")

print(f"S1 VV stack shape: {s1_vv_stack.shape}")
print(f"S1 VH stack shape: {s1_vh_stack.shape}")
print(f"S2 NDVI stack shape: {s2_ndvi_stack.shape}")
print(f"CRS: {s2_ndvi_stack.rio.crs}")

## Step 2.3: Visualize S1 and S2 Time Series (Before Fusion)

In [ ]:
# For point data - visualize multi-sensor time series
if 's1_data' in locals() and 's2_data' in locals():
    fig, axes = plt.subplots(3, 1, figsize=(15, 11))
    
    # Select first location for visualization
    sample_loc = 0
    
    # Get dimension names dynamically
    s1_loc_dim = [d for d in s1_data['VV'].dims if d != 't'][0]
    s2_loc_dim = [d for d in s2_data['NDVI'].dims if d != 't'][0]
    
    # Plot S1 VV
    vv_data = s1_data['VV'].isel({s1_loc_dim: sample_loc})
    axes[0].plot(s1_data.t, vv_data, 'o-', alpha=0.7, color='blue', label='S1 VV')
    axes[0].set_ylabel('VV Backscatter (dB)', fontsize=11)
    axes[0].set_title('Multi-Sensor Time Series (Before Fusion)', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot S1 VH
    vh_data = s1_data['VH'].isel({s1_loc_dim: sample_loc})
    axes[1].plot(s1_data.t, vh_data, 'o-', alpha=0.7, color='purple', label='S1 VH')
    axes[1].set_ylabel('VH Backscatter (dB)', fontsize=11)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot S2 NDVI (with gaps)
    ndvi_data = s2_data['NDVI'].isel({s2_loc_dim: sample_loc})
    axes[2].plot(s2_data.t, ndvi_data, 'o', alpha=0.5, color='gray', 
                 markersize=6, label='S2 NDVI (with cloud gaps)')
    axes[2].set_ylabel('NDVI', fontsize=11)
    axes[2].set_xlabel('Date', fontsize=11)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # Highlight the gap issue
    valid_ndvi = ndvi_data.where(~np.isnan(ndvi_data), drop=True)
    gap_percentage = (1 - len(valid_ndvi) / len(ndvi_data)) * 100
    axes[2].text(0.02, 0.95, f'Cloud gaps: {gap_percentage:.1f}%\nS1 all-weather: 100% coverage', 
                transform=axes[2].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig('s1_s2_raw_timeseries.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nData completeness:")
    print(f"  S1 VV: {(~np.isnan(vv_data)).sum().values}/{len(vv_data)} observations ({(~np.isnan(vv_data)).sum().values/len(vv_data)*100:.1f}%)")
    print(f"  S1 VH: {(~np.isnan(vh_data)).sum().values}/{len(vh_data)} observations ({(~np.isnan(vh_data)).sum().values/len(vh_data)*100:.1f}%)")
    print(f"  S2 NDVI: {(~np.isnan(ndvi_data)).sum().values}/{len(ndvi_data)} observations ({(~np.isnan(ndvi_data)).sum().values/len(ndvi_data)*100:.1f}%)")

## Step 2.4: Prepare Data for MOGPR Fusion

In [ ]:
# For point data - combine S1 and S2 into MOGPR format
if 's1_data' in locals() and 's2_data' in locals():
    print("Preparing data for MOGPR fusion...")
    
    # Merge S1 and S2 datasets
    # Note: They may have different temporal sampling, MOGPR handles this
    combined_data = xr.Dataset({
        'VV': s1_data['VV'],
        'VH': s1_data['VH'],
        'S2ndvi': s2_data['NDVI']  # Must be named 'S2ndvi' for FuseTS convention
    })
    
    print("✓ Combined dataset created")
    print(f"  Variables: {list(combined_data.data_vars)}")
    print(f"  Dimensions: {combined_data.dims}")

# For raster data
if 's1_vv_stack' in locals() and 's2_ndvi_stack' in locals():
    print("Preparing raster data for MOGPR fusion...")
    
    # Combine into single dataset
    combined_data = xr.Dataset({
        'VV': s1_vv_stack,
        'VH': s1_vh_stack,
        'S2ndvi': s2_ndvi_stack
    })
    
    print("✓ Combined raster dataset created")
    print(f"  Variables: {list(combined_data.data_vars)}")
    print(f"  Dimensions: {combined_data.dims}")

## Step 2.5: Apply MOGPR Fusion

MOGPR will:
1. Learn correlations between S1 (VV, VH) and S2 (NDVI)
2. Use S1 data to reconstruct NDVI in cloudy periods
3. Produce gap-free, fused NDVI time series

In [ ]:
print("Applying MOGPR fusion...")
print("This may take a few minutes depending on data size.")
print("MOGPR is learning S1-S2 correlations and reconstructing gaps...\n")

# Initialize MOGPR transformer
mogpr = MOGPRTransformer(
    # You can adjust these parameters for better results
    # kernel_type='RBF',  # Default kernel
    # max_iterations=100  # Default optimization iterations
)

# Fit and transform the combined data
fused_data = mogpr.fit_transform(combined_data)

print("\n✓ MOGPR fusion complete!")
print(f"  Input variables: {list(combined_data.data_vars)}")
print(f"  Output variables: {list(fused_data.data_vars)}")
print(f"  Fused NDVI now gap-free using S1 information!")

# Extract gap-filled NDVI
ndvi_fused = fused_data['S2ndvi']

## Step 2.6: Compare S2-Only vs MOGPR-Fused NDVI

In [ ]:
# For point data - visualize fusion benefit
if 's2_data' in locals():
    fig, axes = plt.subplots(2, 1, figsize=(15, 10))
    
    sample_loc = 0
    
    # Get dimension names
    s2_loc_dim = [d for d in s2_data['NDVI'].dims if d != 't'][0]
    fused_loc_dim = [d for d in ndvi_fused.dims if d != 't'][0]
    
    # Original S2 NDVI (with gaps)
    s2_original = s2_data['NDVI'].isel({s2_loc_dim: sample_loc})
    
    # MOGPR fused NDVI (gap-free)
    ndvi_fused_loc = ndvi_fused.isel({fused_loc_dim: sample_loc})
    
    # Plot 1: Direct comparison
    axes[0].plot(s2_data.t, s2_original, 'o', alpha=0.5, color='gray', 
                 markersize=8, label='S2 NDVI (original, with gaps)')
    axes[0].plot(fused_data.t, ndvi_fused_loc, '-', linewidth=2.5, color='green', 
                 label='MOGPR Fused NDVI (gap-filled using S1)')
    axes[0].set_ylabel('NDVI', fontsize=12)
    axes[0].set_title('MOGPR Fusion Benefit: S1+S2 Gap-Filling', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11, loc='upper right')
    axes[0].grid(True, alpha=0.3)
    
    # Add annotation
    gap_filled = len(ndvi_fused_loc) - (~np.isnan(s2_original)).sum().values
    axes[0].text(0.02, 0.05, f'Gaps filled by MOGPR: {gap_filled} observations\nUsing S1 VV+VH correlations', 
                transform=axes[0].transAxes, fontsize=11, verticalalignment='bottom',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    # Plot 2: S1 VV contribution (show why fusion works)
    vv_loc = s1_data['VV'].isel({[d for d in s1_data['VV'].dims if d != 't'][0]: sample_loc})
    ax2 = axes[1]
    ax2_twin = ax2.twinx()
    
    ax2.plot(s1_data.t, vv_loc, 'o-', alpha=0.6, color='blue', label='S1 VV (all-weather)')
    ax2_twin.plot(fused_data.t, ndvi_fused_loc, '-', linewidth=2, color='green', 
                  label='Fused NDVI')
    
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_ylabel('S1 VV Backscatter (dB)', fontsize=11, color='blue')
    ax2_twin.set_ylabel('NDVI', fontsize=11, color='green')
    ax2.set_title('S1-NDVI Correlation (How MOGPR Learns)', fontsize=13)
    ax2.legend(loc='upper left', fontsize=10)
    ax2_twin.legend(loc='upper right', fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('mogpr_fusion_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Fusion comparison plot saved: mogpr_fusion_comparison.png")

## Step 2.7: Optional - Apply Whittaker Smoothing on Top of MOGPR

In [ ]:
# MOGPR already provides smooth output, but Whittaker can further refine
# This is optional - you can skip this step and use MOGPR output directly

print("Applying additional Whittaker smoothing on MOGPR output...")
print("(This is optional - MOGPR output is already smooth)")

lambda_param = 5000  # Lower lambda since MOGPR already smoothed
ndvi_final = whittaker(ndvi_fused, lmbd=lambda_param)

print("✓ Final smoothed NDVI ready for phenology extraction")

# Alternative: Use MOGPR output directly without additional smoothing
# ndvi_final = ndvi_fused

## Step 2.8: Extract Phenology from Fused NDVI

In [ ]:
print("Extracting phenological metrics from fused NDVI...")

# Extract phenology from gap-filled, fused data
pheno_fused = phenology(
    ndvi_final,
    detection_method='seasonal_amplitude',
    amplitude_threshold=0.2
)

print("✓ Phenology extraction complete (from MOGPR-fused data)")

# Also extract from S2-only for comparison
print("\nExtracting phenology from S2-only NDVI (for comparison)...")
s2_only_smoothed = whittaker(s2_data['NDVI'], lmbd=10000)
pheno_s2only = phenology(
    s2_only_smoothed,
    detection_method='seasonal_amplitude',
    amplitude_threshold=0.2
)

print("✓ Phenology extraction complete (from S2-only data)")
print("\nNow we can compare fusion benefits!")

## Step 2.9: Analyze and Compare Phenology Results

In [ ]:
# Extract phenology arrays from both methods
sos_fused = pheno_fused.da_sos_times.values
eos_fused = pheno_fused.da_eos_times.values
los_fused = pheno_fused.da_los.values
peak_fused = pheno_fused.da_peak_value.values

sos_s2only = pheno_s2only.da_sos_times.values
eos_s2only = pheno_s2only.da_eos_times.values
los_s2only = pheno_s2only.da_los.values
peak_s2only = pheno_s2only.da_peak_value.values

# For point data - create comparison DataFrame
if 's2_data' in locals() and len(sos_fused.shape) >= 1:
    n_locations = sos_fused.shape[1] if len(sos_fused.shape) > 1 else len(sos_fused)
    
    # MOGPR-fused results
    results_fused = pd.DataFrame({
        'location_id': range(n_locations),
        'SOS_fused': sos_fused.flatten()[:n_locations],
        'EOS_fused': eos_fused.flatten()[:n_locations],
        'LOS_fused': los_fused.flatten()[:n_locations],
        'peak_NDVI_fused': peak_fused.flatten()[:n_locations]
    })
    
    # S2-only results
    results_s2only = pd.DataFrame({
        'SOS_s2only': sos_s2only.flatten()[:n_locations],
        'EOS_s2only': eos_s2only.flatten()[:n_locations],
        'LOS_s2only': los_s2only.flatten()[:n_locations],
        'peak_NDVI_s2only': peak_s2only.flatten()[:n_locations]
    })
    
    # Combine
    results_comparison = pd.concat([results_fused, results_s2only], axis=1)
    
    # Calculate differences
    results_comparison['SOS_diff'] = results_comparison['SOS_fused'] - results_comparison['SOS_s2only']
    results_comparison['EOS_diff'] = results_comparison['EOS_fused'] - results_comparison['EOS_s2only']
    results_comparison['LOS_diff'] = results_comparison['LOS_fused'] - results_comparison['LOS_s2only']
    
    # Remove invalid rows
    results_comparison_clean = results_comparison.dropna()
    
    print("\n" + "="*80)
    print("PHENOLOGY COMPARISON: MOGPR Fusion vs S2-Only")
    print("="*80)
    
    print(f"\nValid detections:")
    print(f"  MOGPR-fused: {len(results_comparison_clean[~results_comparison_clean['SOS_fused'].isna()])} locations")
    print(f"  S2-only: {len(results_comparison_clean[~results_comparison_clean['SOS_s2only'].isna()])} locations")
    
    print(f"\nMean differences (MOGPR - S2only):")
    print(f"  SOS: {results_comparison_clean['SOS_diff'].mean():.1f} days (earlier is negative)")
    print(f"  EOS: {results_comparison_clean['EOS_diff'].mean():.1f} days (earlier is negative)")
    print(f"  LOS: {results_comparison_clean['LOS_diff'].mean():.1f} days")
    
    print(f"\nMOGPR-Fused Statistics:")
    print(results_comparison_clean[['SOS_fused', 'EOS_fused', 'LOS_fused', 'peak_NDVI_fused']].describe())
    
    # Export
    results_comparison_clean.to_csv('phenology_mogpr_vs_s2only_comparison.csv', index=False)
    print("\n✓ Comparison results exported to: phenology_mogpr_vs_s2only_comparison.csv")

## Step 2.10: Visualize Phenology on Fused Time Series

In [ ]:
# Visualize phenological markers on fused time series
if 'results_comparison_clean' in locals() and len(results_comparison_clean) > 0:
    
    # Select a location with valid detections in both methods
    valid_both = results_comparison_clean[
        ~results_comparison_clean['SOS_fused'].isna() & 
        ~results_comparison_clean['SOS_s2only'].isna()
    ]
    
    if len(valid_both) > 0:
        sample_loc = int(valid_both.iloc[0]['location_id'])
        
        fig, axes = plt.subplots(2, 1, figsize=(16, 11))
        
        # Get dimension names
        s2_loc_dim = [d for d in s2_data['NDVI'].dims if d != 't'][0]
        fused_loc_dim = [d for d in ndvi_final.dims if d != 't'][0]
        
        # Get time series
        s2_orig = s2_data['NDVI'].isel({s2_loc_dim: sample_loc})
        ndvi_fused_loc = ndvi_final.isel({fused_loc_dim: sample_loc})
        
        # Get phenology for this location
        loc_data = valid_both[valid_both['location_id'] == sample_loc].iloc[0]
        
        # Plot 1: MOGPR-fused phenology
        axes[0].plot(s2_data.t, s2_orig, 'o', alpha=0.3, color='lightgray', 
                     markersize=5, label='S2 NDVI (original)')
        axes[0].plot(fused_data.t, ndvi_fused_loc, '-', linewidth=2.5, color='darkgreen', 
                     label='MOGPR Fused NDVI')
        axes[0].axhline(y=loc_data['peak_NDVI_fused'], color='orange', linestyle='--', 
                       alpha=0.7, linewidth=2, label=f"Peak = {loc_data['peak_NDVI_fused']:.3f}")
        
        axes[0].text(0.02, 0.95, 
                    f"MOGPR-Fused Phenology:\n"
                    f"SOS (DOY): {loc_data['SOS_fused']:.0f}\n"
                    f"EOS (DOY): {loc_data['EOS_fused']:.0f}\n"
                    f"LOS: {loc_data['LOS_fused']:.0f} days", 
                    transform=axes[0].transAxes, fontsize=11, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
        
        axes[0].set_ylabel('NDVI', fontsize=12)
        axes[0].set_title(f'Phenology from MOGPR S1+S2 Fusion - Location {sample_loc}', 
                         fontsize=14, fontweight='bold')
        axes[0].legend(fontsize=11, loc='upper right')
        axes[0].grid(True, alpha=0.3)
        
        # Plot 2: Comparison of phenology detections
        axes[1].plot(s2_data.t, s2_orig, 'o', alpha=0.4, color='gray', 
                     markersize=5, label='S2 NDVI (original)')
        axes[1].plot(fused_data.t, ndvi_fused_loc, '-', linewidth=2, color='green', 
                     label='MOGPR Fused', alpha=0.8)
        
        # Mark SOS and EOS for both methods
        axes[1].axvline(x=loc_data['SOS_fused'], color='darkgreen', linestyle='-', 
                       linewidth=2, alpha=0.7, label=f"SOS (Fused): {loc_data['SOS_fused']:.0f}")
        axes[1].axvline(x=loc_data['SOS_s2only'], color='blue', linestyle='--', 
                       linewidth=2, alpha=0.7, label=f"SOS (S2-only): {loc_data['SOS_s2only']:.0f}")
        
        axes[1].axvline(x=loc_data['EOS_fused'], color='darkred', linestyle='-', 
                       linewidth=2, alpha=0.7, label=f"EOS (Fused): {loc_data['EOS_fused']:.0f}")
        axes[1].axvline(x=loc_data['EOS_s2only'], color='orange', linestyle='--', 
                       linewidth=2, alpha=0.7, label=f"EOS (S2-only): {loc_data['EOS_s2only']:.0f}")
        
        axes[1].set_xlabel('Date', fontsize=12)
        axes[1].set_ylabel('NDVI', fontsize=12)
        axes[1].set_title('Comparison: MOGPR-Fused vs S2-Only Phenology Detection', fontsize=13)
        axes[1].legend(fontsize=9, loc='upper right', ncol=2)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('phenology_mogpr_fused_timeseries.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("\n✓ Phenology visualization saved: phenology_mogpr_fused_timeseries.png")

## Step 2.11: Filter for Paddyfield and Export Results

In [ ]:
# Apply paddyfield filtering to MOGPR-fused results
if 'results_comparison_clean' in locals():
    
    paddyfield_mask = (
        (results_comparison_clean['LOS_fused'] >= 90) & 
        (results_comparison_clean['LOS_fused'] <= 130) &
        (results_comparison_clean['peak_NDVI_fused'] >= 0.6)
    )
    
    paddyfield_fused = results_comparison_clean[paddyfield_mask]
    
    print("\n" + "="*80)
    print("PADDYFIELD FILTERING (MOGPR-Fused Results)")
    print("="*80)
    print(f"Total locations: {len(results_comparison_clean)}")
    print(f"Paddyfield-like: {len(paddyfield_fused)} ({len(paddyfield_fused)/len(results_comparison_clean)*100:.1f}%)")
    
    print(f"\nPaddyfield Statistics (MOGPR-Fused):")
    print(paddyfield_fused[['SOS_fused', 'EOS_fused', 'LOS_fused', 'peak_NDVI_fused']].describe())
    
    # Export final paddyfield phenology
    paddyfield_fused.to_csv('paddyfield_phenology_mogpr_fused.csv', index=False)
    print("\n✓ Paddyfield phenology (MOGPR-fused) exported to: paddyfield_phenology_mogpr_fused.csv")

## Step 2.12: Export Phenology Maps (For Raster Data)

In [ ]:
# Export phenology maps from MOGPR-fused data (if using raster workflow)
if 's1_vv_stack' in locals():
    
    print("Exporting MOGPR-fused phenology maps...")
    
    pheno_fused.da_sos_times.rio.to_raster('mogpr_SOS_map.tif', compress='lzw')
    pheno_fused.da_eos_times.rio.to_raster('mogpr_EOS_map.tif', compress='lzw')
    pheno_fused.da_los.rio.to_raster('mogpr_LOS_map.tif', compress='lzw')
    pheno_fused.da_peak_value.rio.to_raster('mogpr_peak_NDVI_map.tif', compress='lzw')
    pheno_fused.da_season_amplitude.rio.to_raster('mogpr_amplitude_map.tif', compress='lzw')
    
    # Also export S2-only for comparison
    pheno_s2only.da_sos_times.rio.to_raster('s2only_SOS_map.tif', compress='lzw')
    pheno_s2only.da_eos_times.rio.to_raster('s2only_EOS_map.tif', compress='lzw')
    
    print("\n✓ Phenology maps exported:")
    print("  MOGPR-fused: mogpr_*.tif")
    print("  S2-only: s2only_*.tif")

---
## Summary and Key Findings

### What We Accomplished:

1. ✅ **Multi-Sensor Data Acquisition**: Downloaded S1 (VV, VH) + S2 (NDVI) from MPC using sits
2. ✅ **MOGPR Fusion**: Applied Multi-Output Gaussian Process Regression for intelligent gap-filling
3. ✅ **Gap-Free Time Series**: Produced continuous NDVI using S1-S2 correlations
4. ✅ **Phenology Extraction**: Calculated SOS, EOS, LOS from both fused and S2-only data
5. ✅ **Comparison Analysis**: Quantified fusion benefits vs traditional S2-only approach
6. ✅ **Paddyfield Filtering**: Identified rice-specific phenology patterns

### Key Benefits of S1+S2 MOGPR Fusion:

**Gap-Filling Performance**:
- MOGPR fills cloud gaps using S1 all-weather capability
- Produces smoother, more complete time series than S2-only
- Better phenology detection in monsoon/rainy seasons

**Phenology Accuracy**:
- More reliable SOS/EOS detection with continuous data
- Reduced false detections from cloud-contaminated observations
- Better capture of rapid phenological transitions

**Operational Advantages**:
- Works in tropical/cloudy regions where S2-only fails
- Near-real-time monitoring possible with frequent S1 acquisitions
- Suitable for operational paddy monitoring systems

### Outputs Generated:

**CSV Files**:
- `phenology_mogpr_vs_s2only_comparison.csv`: Full comparison of both methods
- `paddyfield_phenology_mogpr_fused.csv`: Final paddyfield phenology metrics

**Visualizations**:
- `s1_s2_raw_timeseries.png`: Multi-sensor input data
- `mogpr_fusion_comparison.png`: Gap-filling demonstration
- `phenology_mogpr_fused_timeseries.png`: Phenology on fused time series

**Maps (if raster workflow)**:
- `mogpr_SOS_map.tif`, `mogpr_EOS_map.tif`, `mogpr_LOS_map.tif`
- Comparison maps: `s2only_*.tif`

### When to Use MOGPR Fusion:

**Recommended for**:
- ✅ Cloudy/tropical regions (Indonesia, SE Asia, Amazon)
- ✅ Monsoon season analysis
- ✅ Operational monitoring requiring complete time series
- ✅ Paddyfield phenology (flooding signal in S1 helps)

**S2-only might suffice for**:
- Clear-sky regions (arid/semi-arid climates)
- Dry season analysis only
- Historical analysis with sufficient cloud-free observations

### Next Steps:

1. **Validation**: Compare MOGPR-fused SOS/EOS with ground truth observations
2. **Multi-year Analysis**: Apply to multiple years to assess inter-annual variability
3. **Crop Type Mapping**: Use phenology metrics as features for paddy classification
4. **Yield Prediction**: Correlate LOS, peak NDVI, and integral with crop yields
5. **Early Warning System**: Set up near-real-time monitoring with S1 frequent revisits

### Parameter Tuning:

- **Lambda (Whittaker)**: Try 5000-10000 for MOGPR output (already smooth)
- **Detection method**: Experiment with `'first_of_slope'` for rapid greening detection
- **Amplitude threshold**: Adjust based on your region's typical NDVI ranges
- **MOGPR kernel**: Try different kernels if default RBF doesn't fit well

---

**For more information:**
- MOGPR methodology: Pipia et al. (2019) - Gaussian Process Regression for crop monitoring
- FuseTS documentation: `/home/unika_sianturi/work/FuseTS/README.md`
- sits + FuseTS workflow: `/home/unika_sianturi/work/FuseTS/SITS_FUSETS_WORKFLOW.md`
- sits documentation: https://e-sensing.github.io/sitsbook/
